In [139]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torch.autograd import Variable

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#import utils
import numpy as np

from PIL import Image

import os, sys

from IPython.display import display
import utils

import itertools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
PATCH_SIZE = 16
ROOT_DIR = 'training/'
IMAGE_DIR = ROOT_DIR + 'images/'

In [82]:
def load_image(filename):
    return mpimg.imread(filename)

def unstack(img):
    return np.array([img[:,:,c] for c in range(3)])

def restack(img):
    return np.stack(img, axis=-1)

In [83]:
files = os.listdir(IMAGE_DIR)

imgs = [load_image(IMAGE_DIR + file) for file in files] # images (400, 400, 3)
imgs_rest = [unstack(im) for im in imgs] # images (3, 400, 400)

In [87]:
X_var = Variable(torch.from_numpy(np.stack(imgs_rest)))

In [73]:
patcher = nn.AvgPool2d(16)

In [74]:
patched = patcher.forward(imgs)

In [149]:
patched.shape

torch.Size([100, 3, 25, 25])

In [144]:
GT_DIR = ROOT_DIR + 'groundtruth/'

gt_imgs = [load_image(GT_DIR + file) for file in files] # images (400, 400)
gt_imgs = [np.reshape(np.array([utils.patch_to_class(p) for p in utils.img_crop(im, PATCH_SIZE, PATCH_SIZE)]), (25, 25))
           for im in gt_imgs]

In [147]:
Y_var = torch.from_numpy(np.stack(np.array(gt_imgs)))

In [148]:
Y_var.shape

torch.Size([100, 25, 25])